In [1]:
import pandas as pd
import numpy as np
import math
import mpmath

In [65]:
#rewards = pd.read_csv('/Users/dulpneto/workspace/OpenAI-GYM-CarRacing-DQN-Risk/r_bound_300/rewards_2.csv')
rewards = pd.read_csv('/Users/dulpneto/workspace/OpenAI-GYM-CarRacing-DQN-Risk/rewards_fixed_policy.log')

lambs = []
for i in range(-15,11):
    lambs.append(i)
    
#for i in range(0,11):
#    lambs.append(round(0.7726278200644+(i/100000000000000), 14))


for lamb in lambs:
    if lamb == 0:
        rewards['{}'.format(lamb)] = rewards.apply(lambda row : row[1]/1000, axis = 1)
        continue
    try:
        rewards['{}'.format(lamb)] = rewards.apply(lambda row : np.sign(lamb) * np.exp(lamb * row[1]), axis = 1)
    except (ValueError, OverflowError) as error:
        print(error)
        print('Error {}'.format(lamb))

rewards

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp


,policy,reward,-15,-14,-13,-12,-11,-10,-9,-8,...,1,2,3,4,5,6,7,8,9,10
0,1,-80.5,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-4.848935e+279,...,1.094698e-35,1.198363e-70,1.311845e-105,1.436074e-140,1.572067e-175,1.720938e-210,1.883907e-245,2.062309e-280,2.257604e-315,0.000000e+00
1,1,-15.7,-1.889516e+102,-2.870299e+95,-4.360173e+88,-6.623391e+81,-1.006137e+75,-1.528388e+68,-2.321722e+61,-3.526849e+54,...,1.519066e-07,2.307561e-14,3.505338e-21,5.324840e-28,8.088783e-35,1.228739e-41,1.866536e-48,2.835392e-55,4.307147e-62,6.542841e-69
2,1,-80.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-8.881134e+277,...,1.804851e-35,3.257489e-70,5.879283e-105,1.061123e-139,1.915170e-174,3.456597e-209,6.238643e-244,1.125982e-278,2.032231e-313,0.000000e+00
3,1,-18.7,-6.600885e+121,-4.992239e+113,-3.775622e+105,-2.855497e+97,-2.159608e+89,-1.633308e+81,-1.235268e+73,-9.342315e+64,...,7.562984e-09,5.719873e-17,4.325931e-25,3.271695e-33,2.474377e-41,1.871368e-49,1.415312e-57,1.070399e-65,8.095407e-74,6.122544e-82
4,1,-15.1,-2.331848e+98,-6.454364e+91,-1.786516e+85,-4.944930e+78,-1.368716e+72,-3.788495e+65,-1.048625e+59,-2.902508e+52,...,2.767919e-07,7.661374e-14,2.120606e-20,5.869665e-27,1.624675e-33,4.496969e-40,1.244725e-46,3.445296e-53,9.536300e-60,2.639570e-66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,5,-25.9,-5.289398e+168,-2.986614e+157,-1.686367e+146,-9.521930e+134,-5.376478e+123,-3.035784e+112,-1.714130e+101,-9.678692e+89,...,5.646417e-12,3.188202e-23,1.800192e-34,1.016463e-45,5.739375e-57,3.240690e-68,1.829829e-79,1.033198e-90,5.833864e-102,3.294042e-113
496,5,-25.9,-5.289398e+168,-2.986614e+157,-1.686367e+146,-9.521930e+134,-5.376478e+123,-3.035784e+112,-1.714130e+101,-9.678692e+89,...,5.646417e-12,3.188202e-23,1.800192e-34,1.016463e-45,5.739375e-57,3.240690e-68,1.829829e-79,1.033198e-90,5.833864e-102,3.294042e-113
497,5,-25.4,-2.925483e+165,-2.723440e+154,-2.535350e+143,-2.360250e+132,-2.197244e+121,-2.045495e+110,-1.904226e+99,-1.772714e+88,...,9.309367e-12,8.666432e-23,8.067899e-34,7.510704e-45,6.991990e-56,6.509100e-67,6.059560e-78,5.641067e-89,5.251477e-100,4.888792e-111
498,5,-25.2,-1.456512e+164,-1.656125e+153,-1.883095e+142,-2.141171e+131,-2.434616e+120,-2.768276e+109,-3.147665e+98,-3.579048e+87,...,1.137049e-11,1.292880e-22,1.470067e-33,1.671538e-44,1.900620e-55,2.161097e-66,2.457273e-77,2.794039e-88,3.176958e-99,3.612356e-110


In [66]:
rewards_by_policy = rewards.groupby('policy')
rewards_by_policy.agg({'reward': ['mean', 'var', 'min', 'max']})

reward                          
          mean          var    min   max
policy                                  
1      -47.207  1860.186920 -212.8 -15.0
2      -18.454     0.472408  -20.4 -17.3
3      -67.026  3455.862954 -307.9 -24.3
4      -25.132     7.642804  -52.2 -24.3
5      -25.885     0.200480  -27.1 -25.2

In [67]:
agg = {}
for lamb in lambs:
    agg['{}'.format(lamb)] = ['mean']
        
def highlight_max(s):
    is_max = s == s.max()
    return ['background: lightgreen' if cell else '' for cell in is_max]        
rewards_by_policy_formated = rewards_by_policy.agg(agg).style.format('{:.2e}').highlight_max(color='lightgreen')
rewards_by_policy_formated
  

,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
policy,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-1.79e+275,-6.85e+182,-2.62e+90,-4.72e-02,7.35e-08,1.58e-14,3.63e-21,8.66e-28,2.13e-34,5.39e-41,1.39e-47,3.65e-54,9.72e-61,2.62e-67
2,-7.84e+130,-1.08e+122,-1.50e+113,-2.07e+104,-2.87e+95,-3.99e+86,-5.57e+77,-7.83e+68,-1.12e+60,-1.66e+51,-2.62e+42,-4.65e+33,-1.01e+25,-2.96e+16,-1.33e+08,-1.85e-02,1.19e-08,1.88e-16,3.50e-24,7.17e-32,1.57e-39,3.61e-47,8.68e-55,2.17e-62,5.62e-70,1.50e-77
3,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-2.74e+265,-5.24e+131,-6.70e-02,4.59e-12,7.78e-23,1.55e-33,3.37e-44,7.81e-55,1.89e-65,4.71e-76,1.20e-86,3.12e-97,8.22e-108
4,-inf,-inf,-5.16e+292,-1.10e+270,-2.35e+247,-5.03e+224,-1.08e+202,-2.30e+179,-4.91e+156,-1.05e+134,-2.24e+111,-4.79e+88,-1.02e+66,-2.19e+43,-4.68e+20,-2.51e-02,1.71e-11,3.31e-22,6.95e-33,1.54e-43,3.54e-54,8.38e-65,2.03e-75,4.98e-86,1.24e-96,3.15e-107
5,-4.61e+174,-8.13e+162,-1.44e+151,-2.57e+139,-4.62e+127,-8.40e+115,-1.55e+104,-2.91e+92,-5.62e+80,-1.13e+69,-2.41e+57,-5.63e+45,-1.50e+34,-4.81e+22,-1.94e+11,-2.59e-02,6.26e-12,4.49e-23,3.51e-34,2.90e-45,2.49e-56,2.21e-67,2.01e-78,1.87e-89,1.78e-100,1.73e-111
